In [50]:
using JuMP, Gurobi, CPLEX
using LinearAlgebra

# Problem setting

- 1 vehicle
- Nodes: Depot, 5 Pickup locations (P1 to P5), and 5 Delivery locations (D1 to D5).
- Time Windows: Randomly generated for each pickup and delivery location.
- Stack Limit: 3 (maximum number of items the vehicle can carry).
- Cost Matrix: Randomly generated costs between each pair of locations.

## Gurobi

In [166]:
# Number of pickup and delivery requests
num_requests = 5
num_nodes = 2 * num_requests + 2 # Includes two depot nodes

# Define the set of pickup nodes P and delivery nodes D
P = 1:num_requests
D = num_requests+1:2*num_requests
depot_start = 2*num_requests+1
depot_end = num_nodes
vehicle_capacity = 3

# Define time windows for each node (here, we're setting random examples)
#time_windows = vcat([(rand(1:10), rand(10:30)) for _ in P], [(rand(11:30), rand(31:40)) for _ in D])
time_windows = vcat([(1, 40) for _ in P], [(1,80) for _ in D])
time_windows = vcat(time_windows, [(0, 20), (40, 100)]) # Time window for the initial depot and the final depot


# Define load sizes for pickup and delivery requests (here, we're setting random examples)
loads = [1 for _ in P] # 1 load for pickups
loads = vcat(loads, -loads) # Loads for deliveries are the negated pickups
loads = vcat(loads, [0, 0]) # Loads for depots are zero

# Random service durations at each location
service_durations = rand(1:4, 2*num_requests) # Service durations between 1 to 4 units
service_durations = vcat(service_durations, [0, 0])

# Define a random symmetric travel time matrix respecting triangular inequality
# 노드의 무작위 위치 생성
node_positions = [rand(0:10, 2) for _ in 1:num_nodes-1]
node_positions = vcat(node_positions, [node_positions[11]]) # 마지막 노드는 11 번째 노드와 동일한 위치에 배치

# 유클리드 거리를 사용하여 이동 시간 계산
function calculate_travel_time(pos1, pos2)
    return round(norm(pos1 - pos2)) # 유클리드 거리를 기반으로 이동 시간 계산
end

# 이동 시간 및 대칭 비용 행렬 생성
travel_time_matrix = zeros(num_nodes, num_nodes)
cost_matrix = zeros(num_nodes, num_nodes)
for i in 1:num_nodes
    for j in 1:num_nodes
        if i != j
            travel_time_matrix[i,j] = calculate_travel_time(node_positions[i], node_positions[j]) * (1/2)
            cost_matrix[i,j] = travel_time_matrix[i,j] * 2 # 비용은 이동 시간에 비례한다고 가정
        end
    end
end

# Define the maximum route duration (here, we're setting a random example)
max_route_duration = rand(50:100)

57

In [167]:
loads

12-element Vector{Int64}:
  1
  1
  1
  1
  1
 -1
 -1
 -1
 -1
 -1
  0
  0

In [168]:
time_windows

12-element Vector{Tuple{Int64, Int64}}:
 (1, 40)
 (1, 40)
 (1, 40)
 (1, 40)
 (1, 40)
 (1, 80)
 (1, 80)
 (1, 80)
 (1, 80)
 (1, 80)
 (0, 20)
 (40, 100)

In [169]:
service_durations

12-element Vector{Int64}:
 2
 3
 3
 1
 3
 4
 4
 3
 2
 1
 0
 0

In [170]:
node_positions

12-element Vector{Vector{Int64}}:
 [0, 5]
 [9, 4]
 [10, 8]
 [7, 5]
 [3, 7]
 [1, 6]
 [3, 1]
 [8, 6]
 [6, 0]
 [9, 3]
 [8, 8]
 [8, 8]

In [171]:
cost_matrix

12×12 Matrix{Float64}:
  0.0  9.0  10.0  7.0  4.0  1.0   5.0  8.0  8.0  9.0  9.0  9.0
  9.0  0.0   4.0  2.0  7.0  8.0   7.0  2.0  5.0  1.0  4.0  4.0
 10.0  4.0   0.0  4.0  7.0  9.0  10.0  3.0  9.0  5.0  2.0  2.0
  7.0  2.0   4.0  0.0  4.0  6.0   6.0  1.0  5.0  3.0  3.0  3.0
  4.0  7.0   7.0  4.0  0.0  2.0   6.0  5.0  8.0  7.0  5.0  5.0
  1.0  8.0   9.0  6.0  2.0  0.0   5.0  7.0  8.0  9.0  7.0  7.0
  5.0  7.0  10.0  6.0  6.0  5.0   0.0  7.0  3.0  6.0  9.0  9.0
  8.0  2.0   3.0  1.0  5.0  7.0   7.0  0.0  6.0  3.0  2.0  2.0
  8.0  5.0   9.0  5.0  8.0  8.0   3.0  6.0  0.0  4.0  8.0  8.0
  9.0  1.0   5.0  3.0  7.0  9.0   6.0  3.0  4.0  0.0  5.0  5.0
  9.0  4.0   2.0  3.0  5.0  7.0   9.0  2.0  8.0  5.0  0.0  0.0
  9.0  4.0   2.0  3.0  5.0  7.0   9.0  2.0  8.0  5.0  0.0  0.0

In [172]:
travel_time_matrix

12×12 Matrix{Float64}:
 0.0  4.5  5.0  3.5  2.0  0.5  2.5  4.0  4.0  4.5  4.5  4.5
 4.5  0.0  2.0  1.0  3.5  4.0  3.5  1.0  2.5  0.5  2.0  2.0
 5.0  2.0  0.0  2.0  3.5  4.5  5.0  1.5  4.5  2.5  1.0  1.0
 3.5  1.0  2.0  0.0  2.0  3.0  3.0  0.5  2.5  1.5  1.5  1.5
 2.0  3.5  3.5  2.0  0.0  1.0  3.0  2.5  4.0  3.5  2.5  2.5
 0.5  4.0  4.5  3.0  1.0  0.0  2.5  3.5  4.0  4.5  3.5  3.5
 2.5  3.5  5.0  3.0  3.0  2.5  0.0  3.5  1.5  3.0  4.5  4.5
 4.0  1.0  1.5  0.5  2.5  3.5  3.5  0.0  3.0  1.5  1.0  1.0
 4.0  2.5  4.5  2.5  4.0  4.0  1.5  3.0  0.0  2.0  4.0  4.0
 4.5  0.5  2.5  1.5  3.5  4.5  3.0  1.5  2.0  0.0  2.5  2.5
 4.5  2.0  1.0  1.5  2.5  3.5  4.5  1.0  4.0  2.5  0.0  0.0
 4.5  2.0  1.0  1.5  2.5  3.5  4.5  1.0  4.0  2.5  0.0  0.0

In [173]:
max_route_duration

57

In [174]:
# Initialize the model
model = Model(Gurobi.Optimizer)

# Define decision variables
@variable(model, x[1:num_nodes, 1:num_nodes], Bin)
@variable(model, service_start_time[1:num_nodes] >=0)
@variable(model, load_after_node[1:num_nodes] >=0)

# Define the objective function to minimize the total cost
@objective(model, Min, sum(cost_matrix[i,j] * x[i,j] for i in 1:num_nodes, j in 1:num_nodes))

# Define the constraints
# Each pickup and delivery match
for i in P
    @constraint(model, sum(x[i, j] for j in num_nodes) == 1)  
    @constraint(model, sum(x[i, j] for j in num_nodes) - sum(x[i+num_requests,j] for j in num_nodes) == 0) 
end

# Leave from and return to the depot
@constraint(model, sum(x[depot_start, j] for j in num_nodes) == 1)  # Leave the depot
@constraint(model, sum(x[i, depot_end] for i in num_nodes) == 1)  # Return to the depot

for i in 1:num_nodes
    @constraint(model, sum(x[j, i] for j in 1:num_nodes) - sum(x[i, j] for j in 1:num_nodes) == 0)
end

# Service time must fall within the time window for each node
for i in 1:num_nodes
    @constraint(model, service_start_time[i] >= time_windows[i][1])
    @constraint(model, service_start_time[i] <= time_windows[i][2])
end

# Service time constraints
for i in 1:num_nodes
    for j in 1:num_nodes
        @constraint(model, service_start_time[j] >= (service_start_time[i] + service_durations[i] + travel_time_matrix[i,j]) * x[i,j])
    end
end

# Service time constraints
for i in P
    @constraint(model, service_start_time[i+num_requests] - service_start_time[i] - service_durations[i] - travel_time_matrix[i,i+num_requests] >= 0)
end

# Total Route time duration constraint
@constraint(model, service_start_time[depot_end] - service_start_time[depot_start] <= max_route_duration)

# Vehicle load and capacity constraints
for i in 1:num_nodes
    for j in 1:num_nodes
        if i != j
            @constraint(model, load_after_node[j] >= (load_after_node[i] + loads[j]) * x[i,j])
        end
    end
end

for i in 1:num_nodes
    @constraint(model, load_after_node[i] <= min(vehicle_capacity, vehicle_capacity+loads[i]))
    @constraint(model, load_after_node[i] >= max(0, loads[i]))
end

# Solve the problem
optimize!(model)

# Check if the model has a solution
if termination_status(model) == MOI.OPTIMAL || termination_status(model) == MOI.LOCALLY_SOLVED
    println("An optimal solution has been found:")
    
    # Print the route
    println("Vehicle route:")
    for i in 1:num_nodes
        for j in 1:num_nodes
            if value(x[i,j]) > 0.9
                println("Node $i to node $j")
            end
        end
    end
    
    # Print the service start times
    println("\nService start times:")
    for i in 1:2*
        println("Node $i: ", value(service_time[i]))
    end
    
    # Print the vehicle loads
    println("\nVehicle loads after each node:")
    for i in 1:num_nodes
        println("Node $i: ", value(vehicle_load[i]))
    end
else
    println("No optimal solution found. Status: ", termination_status(model))
end


Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-21
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: AMD Ryzen 9 5900X 12-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 78 rows, 168 columns and 341 nonzeros
Model fingerprint: 0xbab19aad
Model has 276 quadratic constraints
Variable types: 24 continuous, 144 integer (144 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 9e+00]
  Objective range  [1e+00, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 61 rows and 5 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 24 available processors)

Solution count 0

Model is infeasible or unbounded
Best objective -, best bound -, gap -

User-

## Dynanic Programming

In [9]:


# Define time windows for each node (example)
time_windows = vcat(time_windows, [(rand(1:10), rand(11:20)) for _ in P], [(rand(21:30), rand(31:40)) for _ in D])
time_windows = vcat(time_windows, [(0,0),(40,40)]) # Depot

# Define load sizes for pickup and delivery requests
loads = [1 for _ in P] # Assuming a load size of 1 for simplicity
loads = vcat(loads, -loads) # Delivery loads are negative

# Define a symmetric cost matrix
cost_matrix = [rand(1:10) for i in 1:num_nodes, j in 1:num_nodes]
for i in 1:num_nodes
    for j in i:num_nodes
        cost_matrix[i,j] = cost_matrix[j,i] = (i == j) ? 0 : cost_matrix[i,j]
    end
end

# Define a symmetric travel time matrix respecting triangular inequality
travel_time_matrix = cost_matrix # For simplicity, using the same values as cost

# Initialize the model
model = Model(Gurobi.Optimizer)

# Define decision variables
@variable(model, x[1:num_nodes, 1:num_nodes], Bin)
@variable(model, service_time[1:num_nodes])
@variable(model, vehicle_load[1:num_nodes])

# Define the objective function to minimize the total cost
@objective(model, Min, sum(cost_matrix[i,j] * x[i,j] for i in 1:num_nodes, j in 1:num_nodes))

# Constraints
# Each pickup node must be visited exactly once
for i in P
    @constraint(model, sum(x[i,j] for j in [depot; D]) == 1)
end

# Flow conservation for the vehicle
@constraint(model, sum(x[depot, j] for j in P) == 1)
@constraint(model, sum(x[i, depot] for i in D) == 1)

# Service time must fall within the time window for each node
for i in 1:num_nodes
    @constraint(model, service_time[i] >= time_windows[i][1])
    @constraint(model, service_time[i] <= time_windows[i][2])
end

# The vehicle's load must not exceed its capacity after each pickup or delivery
@constraint(model, vehicle_load[depot] == 0)
for i in 1:num_nodes
    for j in 1:num_nodes
        if i != j
            @constraint(model, vehicle_load[j] >= vehicle_load[i] + loads[i] * x[i,j])
        end
    end
end

# Solve the problem
optimize!(model)

# Check if the model has a solution
if termination_status(model) == MOI.OPTIMAL || termination_status(model) == MOI.LOCALLY_SOLVED
    println("An optimal solution has been found:")
    
    # Print the route
    println("Vehicle route:")
    for i in 1:num_nodes
        for j in 1:num_nodes
            if value(x[i,j]) > 0.9
                println("Node $i to node $j")
            end
        end
    end
    
    # Print the service start times
    println("\nService start times:")
    for i in 1:num_nodes
        println("Node $i: ", value(service_time[i]))
    end
    
    # Print the vehicle loads
    println("\nVehicle loads after each node:")
    for i in 1:num_nodes
        println("Node $i: ", value(vehicle_load[i]))
    end
else
    println("No optimal solution found. Status: ", termination_status(model))
end


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-21


LoadError: BoundsError: attempt to access 10-element Vector{Int64} at index [11]